# Project (P2)

## Getting familiar with the data

In [196]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

DATA_FOLDER = 'data/'
WIKISPEEDIA_PATHS = DATA_FOLDER + "wikispeedia_paths-and-graph/"

ARTICLES = WIKISPEEDIA_PATHS + "articles.tsv"
CATEGORIES = WIKISPEEDIA_PATHS + "categories.tsv"
LINKS = WIKISPEEDIA_PATHS + "links.tsv"
PATHS_FINISHED = WIKISPEEDIA_PATHS + "paths_finished.tsv"
PATHS_UNFINISHED = WIKISPEEDIA_PATHS + "paths_unfinished.tsv"
SHORTEST_PATHS = WIKISPEEDIA_PATHS + "shortest-path-distance-matrix.txt"

FEMALES = DATA_FOLDER + "listwomen.txt"
ALL_PATHS = DATA_FOLDER + "all_paths_overview.csv"

In [197]:
articles = pd.read_csv(ARTICLES, sep='\t', names=["article"], skiprows = 12)
categories = pd.read_csv(CATEGORIES, sep='\t', names=["article", "category"], skiprows = 13)
links = pd.read_csv(LINKS, sep='\t', names=["link_source", "link_target"], skiprows = 12)
paths_finished = pd.read_csv(PATHS_FINISHED, sep='\t', names=["hashedIpAddress", "timestamp", "durationInSec", "path", "rating"], skiprows = 15)
paths_unfinished = pd.read_csv(PATHS_UNFINISHED, sep='\t', names=["hashedIpAddress", "timestamp", "durationInSec", "path", "target", "type"], skiprows = 17)

paths = paths_finished["path"].str.split(";", n = 1, expand = True)


paths_finished['source'] = paths_finished['path'].str.split(';').str[0]
paths_unfinished['source'] = paths_unfinished['path'].str.split(';').str[0]
paths_finished['target'] = paths_finished['path'].str.split(';').str[-1]

(paths_finished.groupby('source')['path'].count()).loc[lambda x: x > 1].sort_values(ascending = False)


source
Brain                                  1092
Asteroid                               1059
Theatre                                 931
Pyramid                                 674
Batman                                  602
                                       ... 
Moore%27s_Law                             2
Thomas_More                               2
Political_integration_of_India            2
William_Thomson%2C_1st_Baron_Kelvin       2
Starship_Troopers                         2
Name: path, Length: 4028, dtype: int64

In [204]:
women = pd.read_csv(FEMALES, sep=',', names=['Name', 'Subject'], skiprows=1)
women.head()


womenName = women['Name']
people = categories.loc[list(map(lambda x: x.startswith('subject.People.'), categories['category']))]
menName = people[~people['article'].isin(womenName)]['article']

paths_women_target = paths_finished[paths_finished['target'].isin(womenName)]
#paths_women_target = paths_women_target.merge(categories, left_on=['target'], right_on=['article'], how='inner')

unpaths_women_target = paths_unfinished[paths_unfinished['target'].isin(womenName)]
#unpaths_women_target = unpaths_women_target.merge(categories, left_on=['target'], right_on=['article'], how='inner')



unpaths_men_target = paths_unfinished[paths_unfinished['target'].isin(menName)]
#unpaths_men_target = unpaths_men_target.merge(categories, left_on=['target'], right_on=['article'], how='inner')

paths_men_target = paths_finished[paths_finished['target'].isin(menName)]
#paths_men_target = paths_men_target.merge(categories, left_on=['target'], right_on=['article'], how='inner')
#paths_men_target
paths_women_target

combinations = paths_women_target.merge(paths_men_target, on=['source'], how='inner')
combinations_uf = unpaths_women_target.merge(paths_men_target, on=['source'], how='inner')
combinations_um = paths_women_target.merge(unpaths_men_target, on=['source'], how='inner')
#uncombinations = unpaths_women_target.merge(unpaths_men_target, on=['source'], how='inner')
#df = combinations[combinations['category_x'] == combinations['category_y']]
#df
combinations[['target_x','target_y']]

,target_x,target_y
0,Elizabeth_I_of_England,John_F._Kennedy
1,Elizabeth_I_of_England,John_F._Kennedy
2,Elizabeth_I_of_England,Henry_David_Thoreau
3,Elizabeth_I_of_England,John_F._Kennedy
4,Elizabeth_I_of_England,John_F._Kennedy
...,...,...
469,Anne_Frank,Henry_IV_of_England
470,Miranda_Otto,Socrates
471,Bette_Davis,Ronald_Reagan
472,Bette_Davis,Ronald_Reagan


In [199]:
## All_paths_overview

all_paths_overview = pd.read_csv(ALL_PATHS)

all_paths_overview



,Unnamed: 0,source,target,human_length,finished,failure
0,0,14th_century,African_slave_trade,9,True,NaN
1,1,14th_century,African_slave_trade,5,True,NaN
2,2,14th_century,African_slave_trade,8,True,NaN
3,3,14th_century,Greece,4,True,NaN
4,4,14th_century,John_F._Kennedy,7,True,NaN
...,...,...,...,...,...,...
76188,24870,Franz_Kafka,Cholera,8,False,Ghana
76189,24871,Modern_history,Hollandic,1,False,Modern_history
76190,24872,Computer_programming,The_Beatles,5,False,<
76191,24873,Jamaica,Alan_Turing,4,False,Battle_of_Midway


In [200]:
## Shortest distance matrix. 

shortest_distance_text = np.loadtxt(SHORTEST_PATHS, dtype=str, comments="#", unpack=False)
shortest_distance_matrix = []
for row in shortest_distance_text:
    row_formated = [] 
    for c in row:
        if c.isdigit(): #Might want to do with safe casting instead
            row_formated.append(int(c))
        else:
            row_formated.append(-1)
        
    shortest_distance_matrix.append(np.array(row_formated))

shortest_distance_matrix = np.array(shortest_distance_matrix)

all_paths_overview.head()

,Unnamed: 0,source,target,human_length,finished,failure
0,0,14th_century,African_slave_trade,9,True,NaN
1,1,14th_century,African_slave_trade,5,True,NaN
2,2,14th_century,African_slave_trade,8,True,NaN
3,3,14th_century,Greece,4,True,NaN
4,4,14th_century,John_F._Kennedy,7,True,NaN


In [201]:
all_paths_overview

,Unnamed: 0,source,target,human_length,finished,failure
0,0,14th_century,African_slave_trade,9,True,NaN
1,1,14th_century,African_slave_trade,5,True,NaN
2,2,14th_century,African_slave_trade,8,True,NaN
3,3,14th_century,Greece,4,True,NaN
4,4,14th_century,John_F._Kennedy,7,True,NaN
...,...,...,...,...,...,...
76188,24870,Franz_Kafka,Cholera,8,False,Ghana
76189,24871,Modern_history,Hollandic,1,False,Modern_history
76190,24872,Computer_programming,The_Beatles,5,False,<
76191,24873,Jamaica,Alan_Turing,4,False,Battle_of_Midway


In [202]:
def get_articles_indexes(article_names):
    indexes = []
    for article in article_names.array:
        indexes.append(articles.index[articles['article'] == article].tolist()[0])
    return tuple(indexes)


# Paths with target articles that doesn't exist are deleted. 
condition = ~(all_paths_overview['target'].isin(articles['article'].array))
all_paths_overview = all_paths_overview.drop(all_paths_overview[condition].index)
#shortest_distance_matrix[(0,0,1),(0,1,1)] How to access the matrix

all_paths_overview['shortest_path'] = shortest_distance_matrix[get_articles_indexes(all_paths_overview['source']), get_articles_indexes(all_paths_overview['target'])]
all_paths_cleaned = all_paths_overview.drop(all_paths_overview[all_paths_overview['shortest_path'] < 0].index)
all_paths_overview                                                                              

,Unnamed: 0,source,target,human_length,finished,failure,shortest_path
0,0,14th_century,African_slave_trade,9,True,NaN,3
1,1,14th_century,African_slave_trade,5,True,NaN,3
2,2,14th_century,African_slave_trade,8,True,NaN,3
3,3,14th_century,Greece,4,True,NaN,2
4,4,14th_century,John_F._Kennedy,7,True,NaN,3
...,...,...,...,...,...,...,...
76188,24870,Franz_Kafka,Cholera,8,False,Ghana,3
76189,24871,Modern_history,Hollandic,1,False,Modern_history,3
76190,24872,Computer_programming,The_Beatles,5,False,<,3
76191,24873,Jamaica,Alan_Turing,4,False,Battle_of_Midway,3


In [203]:
all_paths_cleaned

,Unnamed: 0,source,target,human_length,finished,failure,shortest_path
0,0,14th_century,African_slave_trade,9,True,NaN,3
1,1,14th_century,African_slave_trade,5,True,NaN,3
2,2,14th_century,African_slave_trade,8,True,NaN,3
3,3,14th_century,Greece,4,True,NaN,2
4,4,14th_century,John_F._Kennedy,7,True,NaN,3
...,...,...,...,...,...,...,...
76188,24870,Franz_Kafka,Cholera,8,False,Ghana,3
76189,24871,Modern_history,Hollandic,1,False,Modern_history,3
76190,24872,Computer_programming,The_Beatles,5,False,<,3
76191,24873,Jamaica,Alan_Turing,4,False,Battle_of_Midway,3


In [214]:

## Don't do like this, create the shortest path before doing the women and men target slices
paths_women_target['shortest_path'] = shortest_distance_matrix[get_articles_indexes(paths_women_target['source']), get_articles_indexes(paths_women_target['target'])]
paths_men_target['shortest_path'] = shortest_distance_matrix[get_articles_indexes(paths_men_target['source']), get_articles_indexes(paths_men_target['target'])]

#paths_women_target

combinations = paths_women_target.merge(paths_men_target, on=['source','shortest_path'], how='inner')

combinations[['target_x','target_y','shortest_path']]

<ipython-input-214-649073555749>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paths_women_target['shortest_path'] = shortest_distance_matrix[get_articles_indexes(paths_women_target['source']), get_articles_indexes(paths_women_target['target'])]
<ipython-input-214-649073555749>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paths_men_target['shortest_path'] = shortest_distance_matrix[get_articles_indexes(paths_men_target['source']), get_articles_indexes(paths_men_target['target'])]


,target_x,target_y,shortest_path
0,Natalie_Portman,John_Logie_Baird,3
1,Natalie_Portman,John_Logie_Baird,3
2,Natalie_Portman,John_Logie_Baird,3
3,Natalie_Portman,John_Logie_Baird,3
4,Natalie_Portman,John_Logie_Baird,3
...,...,...,...
213,Cleopatra_VII,Joseph_Stalin,3
214,Cleopatra_VII,Joseph_Stalin,3
215,Anne_Frank,Adolf_Hitler,3
216,Anne_Frank,Adolf_Hitler,3


### **Articles**
Contains the name of the articles

In [18]:
print(f"Number of articles in the dataset: {len(articles)}")
articles.head(10)

Number of articles in the dataset: 4604


,article
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1,%C3%85land
2,%C3%89douard_Manet
3,%C3%89ire
4,%C3%93engus_I_of_the_Picts
5,%E2%82%AC2_commemorative_coins
6,10th_century
7,11th_century
8,12th_century
9,13th_century


### Categories

Every row contains an article and its corresponding category. 

In [19]:
print(f"Number of rows in the dataset: {len(categories)}")
categories.head()

Number of rows in the dataset: 5204


,article,category
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.History.British_History.British_Histor...
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.People.Historical_figures
2,%C3%85land,subject.Countries
3,%C3%85land,subject.Geography.European_Geography.European_...
4,%C3%89douard_Manet,subject.People.Artists


In [32]:
# Number of articles in each category
n_artcat = categories.groupby(categories["category"]).nunique()

print(f"The number of categories is {len(n_artcat)}")
n_artcat

The number of categories is 129


,article
category,
subject.Art.Art,36
subject.Art.Artists,2
subject.Business_Studies.Business,28
subject.Business_Studies.Companies,18
subject.Business_Studies.Currency,14
...,...
subject.Science.Physics.Electricity_and_Electronics,20
subject.Science.Physics.General_Physics,55
subject.Science.Physics.Space_Astronomy,105


In [43]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(n_artcat)

                                                    article
category                                                   
subject.Art.Art                                          36
subject.Art.Artists                                       2
subject.Business_Studies.Business                        28
subject.Business_Studies.Companies                       18
subject.Business_Studies.Currency                        14
subject.Business_Studies.Economics                       28
subject.Citizenship.Animal_and_Human_Rights              17
subject.Citizenship.Community_organisations              11
subject.Citizenship.Conflict_and_Peace                   10
subject.Citizenship.Culture_and_Diversity                16
subject.Citizenship.Education                            17
subject.Citizenship.Environment                          26
subject.Citizenship.Law                                  19
subject.Citizenship.Media                                11
subject.Citizenship.Politics_and_governm

**Actors_models_and_celebrities**

In [130]:
# Add underscore inbetween first and last names in female_actors dataframe
female_actors.women_label = list(map(lambda x: x.replace(' ', '_'), female_actors.women_label))
female_celeb.women_label = list(map(lambda x: x.replace(' ', '_'), female_celeb.women_label))
female_model.women_label = list(map(lambda x: x.replace(' ', '_'), female_model.women_label))

In [134]:
female_actors.head()

,women,women_label
0,http://www.wikidata.org/entity/Q230739,Katie_Couric
1,http://www.wikidata.org/entity/Q449014,Christine_Chubbuck
2,http://www.wikidata.org/entity/Q494466,Andrea_Mitchell
3,http://www.wikidata.org/entity/Q153576,Linh_Nga
4,http://www.wikidata.org/entity/Q79845,Madhu_Shalini


In [95]:
#female_actors = female_actors["women_label"]

In [114]:
# Let's first take a look at the category subject.People.Actors_models_and_celebrities
wikispeedia_actors = categories[categories["category"] == "subject.People.Actors_models_and_celebrities"]
wikispeedia_actors.head()

,article,category
276,Andrew_Robinson,subject.People.Actors_models_and_celebrities
638,Bette_Davis,subject.People.Actors_models_and_celebrities
760,Brandon_Routh,subject.People.Actors_models_and_celebrities
787,Britney_Spears,subject.People.Actors_models_and_celebrities
793,Bruce_Lee,subject.People.Actors_models_and_celebrities


In [135]:
act = wikispeedia_actors.merge(female_actors, left_on='article', right_on='women_label')
cele = wikispeedia_actors.merge(female_celeb, left_on='article', right_on='women_label')
mod = wikispeedia_actors.merge(female_model, left_on='article', right_on='women_label')

In [136]:
act

,article,category,women,women_label
0,Bette_Davis,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q71206,Bette_Davis
1,Emma_Roberts,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q228598,Emma_Roberts
2,Evan_Rachel_Wood,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q229230,Evan_Rachel_Wood
3,Jane_Fonda,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q41142,Jane_Fonda
4,Keira_Knightley,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q42581,Keira_Knightley
5,Miranda_Otto,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q294975,Miranda_Otto
6,Natalie_Portman,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q37876,Natalie_Portman
7,Sharon_Tate,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q228699,Sharon_Tate
8,Vivien_Leigh,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q129429,Vivien_Leigh


In [137]:
cele

,article,category,women,women_label
0,Britney_Spears,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q11975,Britney_Spears


In [138]:
mod

,article,category,women,women_label
0,Britney_Spears,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q11975,Britney_Spears
1,Emma_Roberts,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q228598,Emma_Roberts
2,Evan_Rachel_Wood,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q229230,Evan_Rachel_Wood
3,Sharon_Tate,subject.People.Actors_models_and_celebrities,http://www.wikidata.org/entity/Q228699,Sharon_Tate


In [117]:
wikispeedia_actors

,article,category
276,Andrew_Robinson,subject.People.Actors_models_and_celebrities
638,Bette_Davis,subject.People.Actors_models_and_celebrities
760,Brandon_Routh,subject.People.Actors_models_and_celebrities
787,Britney_Spears,subject.People.Actors_models_and_celebrities
793,Bruce_Lee,subject.People.Actors_models_and_celebrities
971,Charlie_Chaplin,subject.People.Actors_models_and_celebrities
1017,Christian_Bale,subject.People.Actors_models_and_celebrities
1278,Daniel_Day-Lewis,subject.People.Actors_models_and_celebrities
1344,Diane_Keaton,subject.People.Actors_models_and_celebrities
1547,Emma_Roberts,subject.People.Actors_models_and_celebrities


**TACKLE ALL PEOPLE CATEGORIES**

In [62]:
df = categories
df['categories_people'] = list(
    map(lambda x: x.startswith('subject.People.'), df['category'])) 

In [65]:
ca = df[df["categories_people"] == True]
ca

,article,category,categories_people
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.People.Historical_figures,True
4,%C3%89douard_Manet,subject.People.Artists,True
8,%C3%93engus_I_of_the_Picts,subject.People.Historical_figures,True
67,A._E._J._Collins,subject.People.Sports_and_games_people,True
88,Abbas_I_of_Persia,subject.People.Historical_figures,True
...,...,...,...
5188,Zhang_Qian,subject.People.Historical_figures,True
5190,Zheng_He,subject.People.Historical_figures,True
5191,Ziad_Jarrah,subject.People.Historical_figures,True
5197,Zionism,subject.People.Political_People,True


### Links

Contains the source and target article that the dataset contains.

In [34]:
print(f"Number of combinations of source and target articles in the dataset: {len(links)}")
links.head()

Number of combinations of source and target articles in the dataset: 119882


,link_source,link_target
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland


### Finished paths

In [36]:
print(f"Number of finished paths: {len(paths_finished)}")
paths_finished.head(5)

Number of finished paths: 51318


,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0


### Unfinished paths

In [38]:
print(f"Number of finished paths: {len(paths_unfinished)}")
paths_unfinished.head(5)

Number of finished paths: 24875


,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout
